__качаем объявления о продаже недвижимости с avito.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sevastopol/kvartiry/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sevastopol/zemelnye_uchastki/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sevastopol/doma_dachi_kottedzhi/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sudak/kvartiry/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sudak/zemelnye_uchastki/prodam',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sudak/doma_dachi_kottedzhi/prodam',
    },
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://www.avito.ru/bahchisaray/kvartiry/prodam',
#     }, 
    {
            'place': 'bahchisaray',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/bahchisaray/zemelnye_uchastki/prodam',
    },{
            'place': 'bahchisaray',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam',
    },

]

---

In [4]:
# from selenium import webdriver
# from lib.avito import AvitoPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = AvitoPageParser().transform(browser)
# # browser.close()

In [5]:
# browser.close()

In [6]:
# data

---

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.avito import AvitoParser

In [8]:
dest_folder = 'data/2024/raw'

In [ ]:
%%time 


options = webdriver.FirefoxOptions()
#options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    sleep(3)
    
    data = AvitoParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place

    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/avito/avito_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )

browser.close()

[INFO    ] 2024-09-24 10:32:29 | open virtual browser


'https://www.avito.ru/sevastopol/kvartiry/prodam'

[INFO    ] 2024-09-24 10:32:40 | page 1: 53 items
[INFO    ] 2024-09-24 10:32:53 | page 2: 50 items
[INFO    ] 2024-09-24 10:33:04 | page 3: 50 items
[INFO    ] 2024-09-24 10:33:15 | page 4: 50 items
[INFO    ] 2024-09-24 10:33:24 | page 5: 50 items
[INFO    ] 2024-09-24 10:33:34 | page 6: 50 items
[INFO    ] 2024-09-24 10:33:44 | page 7: 50 items
[INFO    ] 2024-09-24 10:33:54 | page 8: 50 items
[INFO    ] 2024-09-24 10:34:04 | page 9: 50 items
[INFO    ] 2024-09-24 10:34:14 | page 10: 49 items
[INFO    ] 2024-09-24 10:34:25 | page 11: 50 items
[INFO    ] 2024-09-24 10:34:35 | page 12: 49 items
[INFO    ] 2024-09-24 10:34:46 | page 13: 49 items
[INFO    ] 2024-09-24 10:34:57 | page 14: 50 items
[INFO    ] 2024-09-24 10:35:08 | page 15: 50 items
[INFO    ] 2024-09-24 10:35:21 | page 16: 50 items
[INFO    ] 2024-09-24 10:35:31 | page 17: 50 items
[INFO    ] 2024-09-24 10:35:42 | page 18: 50 items
[INFO    ] 2024-09-24 10:35:54 | page 19: 50 items
[INFO    ] 2024-09-24 10:36:04 | page 20

In [ ]:
# browser.close()

In [ ]:
!date